In [17]:
import os
import argparse
import numpy as np
import pandas as pd

import torch
from torch import optim
import torch.nn as nn

from torch.utils.data import DataLoader
import torch.nn.functional as F

import torchvision.datasets as datasets
from torchvision import transforms
from data_loader import *
from model import *
from query import *

In [18]:
DEFAULT_DATASET_DIR = "./dataset/"
DEFAULT_BATCH_SIZE = 64
DEFAULT_NO_EPOCHS = 50
DEFAULT_MODEL_PATH = "./ckpt/exp_4/model_ckpt.tar"
config = Config(DEFAULT_DATASET_DIR, DEFAULT_BATCH_SIZE, DEFAULT_NO_EPOCHS)

In [3]:
images_folder_dataset = datasets.ImageFolder(root=config.images_dir+'signatures/valid')
siamese_dataset = SignatureDataset(dataset=images_folder_dataset)
print(len(siamese_dataset))

144


In [4]:
img0_file = siamese_dataset.genuine[0][1]
img1_file = siamese_dataset.genuine[0][1]
img0 = Image.open(img0_file).convert("L")
img1 = Image.open(img1_file).convert("L")
 
img0_file, img1_file

('./dataset/signatures/valid\\full_org\\original_50_1.png',
 './dataset/signatures/valid\\full_org\\original_50_1.png')

In [5]:
genuine_sigs = siamese_dataset.genuine
forged_sigs = siamese_dataset.forged

In [6]:
len(siamese_dataset)

144

In [3]:
query_model = QueryModel(DEFAULT_MODEL_PATH)

D:\Codes\axis-ai-challenge\model\model.py:72: UserWarning: nn.init.kaiming_normal is now deprecated in favor of nn.init.kaiming_normal_.
  nn.init.kaiming_normal(m.weight, mode='fan_in')


In [8]:
output = query_model.predict(img0, img1)

In [16]:
print(output.item())
log_probas = F.sigmoid(output)
print(log_probas.item())

4.145488739013672
0.9844112396240234


In [10]:
img0_file = siamese_dataset.genuine[52][1]
img1_file = siamese_dataset.forged[53][1]
img0 = Image.open(img0_file).convert("L")
img1 = Image.open(img1_file).convert("L")
 
img0_file, img1_file

('./dataset/signatures/valid\\full_org\\original_52_13.png',
 './dataset/signatures/valid\\full_forg\\forgeries_52_14.png')

In [11]:
output = query_model.predict(img0, img1)

In [15]:
print(output.item())
log_probas = F.sigmoid(output)
print(log_probas.item())

4.145488739013672
0.9844112396240234


In [13]:
img0_file = 'vin_real.jpg'
img1_file = 'vin_forged.jpg'
img0 = Image.open(img0_file).convert("L")
img1 = Image.open(img1_file).convert("L")
 
img0_file, img1_file

('vin_real.jpg', 'vin_forged.jpg')

In [14]:
output = query_model.predict(img0, img1)

In [13]:
print(output.item())
log_probas = F.sigmoid(output)
print(log_probas.item())

4.145488739013672
0.9844112396240234


# CEDAR Dataset Scores

In [16]:
images_folder_dataset = datasets.ImageFolder(root=config.images_dir+'signatures/valid')
siamese_dataset = SignatureDataset(dataset=images_folder_dataset)

In [17]:
len(siamese_dataset)

144

In [18]:
total = len(siamese_dataset)
tp = 0
tn = 0
fp = 0
fn = 0
for x1, x2, label in siamese_dataset:
    print(label)
    print()
    x1 = x1.unsqueeze_(0).cuda()
    x2 = x2.unsqueeze_(0).cuda()
    x1, x2 = Variable(x1, volatile=True), Variable(x2, volatile=True)

    batch_size = x1.shape[0]

    # compute log probabilities
    output = query_model.model(x1, x2)
    log_probas = F.sigmoid(output)
    print(output.item())
    log_probas = F.sigmoid(output)
    print(log_probas.item())
    
    if log_probas >= 0.5 and label == 1:
        tp += 1
    elif log_probas <= 0.5 and label == 0:
        tn += 1
    elif log_probas >= 0.5 and label == 0:
        fp += 1
    elif log_probas <= 0.5 and label == 1:
        fn += 1

    print()

tensor([ 0.])



C:\Users\arush\Anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:11: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # This is added back by InteractiveShellApp.init_path()


-1.6451008319854736
0.16177219152450562

tensor([ 0.])

-1.9239610433578491
0.12742051482200623

tensor([ 0.])

-2.236957550048828
0.09648043662309647

tensor([ 0.])

-3.095597505569458
0.043289221823215485

tensor([ 0.])

-2.0741066932678223
0.111639104783535

tensor([ 0.])

-1.2641125917434692
0.22026674449443817

tensor([ 1.])

0.13223516941070557
0.533010721206665

tensor([ 1.])

1.132020354270935
0.7562115788459778

tensor([ 1.])

-0.8279778957366943
0.3040727972984314

tensor([ 1.])

-0.551814079284668
0.365443617105484

tensor([ 0.])

-1.8028827905654907
0.14150051772594452

tensor([ 1.])

0.39658403396606445
0.5978666543960571

tensor([ 1.])

0.3688143193721771
0.591172456741333

tensor([ 1.])

0.5712929964065552
0.6390615105628967

tensor([ 1.])

1.5481083393096924
0.8246403932571411

tensor([ 0.])

-1.6525135040283203
0.160769522190094

tensor([ 1.])

-0.6597203016281128
0.3408024311065674

tensor([ 0.])

-0.2664864957332611
0.4337698817253113

tensor([ 1.])

1.02095913887023

In [8]:
correct = tp+tn
print(100*correct/total)

print((tp/total)*100, (fp/total)*100)
print((tn/total)*100, (fn/total)*100)


74.77876106194691
50.442477876106196 20.79646017699115
24.336283185840706 4.424778761061947


# Sample Dataset from Axis AI Scores

In [4]:
images_folder_dataset = datasets.ImageFolder(root=config.images_dir+'data_complete/sample_Signature/train')
siamese_dataset = SignatureDataset(dataset=images_folder_dataset, dataset_type='sample')

In [5]:
len(siamese_dataset)

226

In [6]:
total = len(siamese_dataset)
tp = 0
tn = 0
fp = 0
fn = 0
for x1, x2, label in siamese_dataset:
    print(label)
    print()
    x1 = x1.unsqueeze_(0).cuda()
    x2 = x2.unsqueeze_(0).cuda()
    x1, x2 = Variable(x1, volatile=True), Variable(x2, volatile=True)

    batch_size = x1.shape[0]

    # compute log probabilities
    output = query_model.model(x1, x2)
    log_probas = F.sigmoid(output)
    print(output.item())
    log_probas = F.sigmoid(output)
    print(log_probas.item())
    
    if log_probas >= 0.5 and label == 1:
        tp += 1
        print("TRUE POSITIVE")
    elif log_probas <= 0.5 and label == 0:
        tn += 1
        print("TRUE NEGATIVE")
    elif log_probas >= 0.5 and label == 0:
        fp += 1
        print("FALSE POSITIVE")
    elif log_probas <= 0.5 and label == 1:
        fn += 1
        print("FALSE NEGATIVE")

    print()

tensor([ 1.])



C:\Users\arush\Anaconda3\envs\tf-gpu\lib\site-packages\ipykernel_launcher.py:11: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  # This is added back by InteractiveShellApp.init_path()


2.1727776527404785
0.8977780938148499
TRUE POSITIVE

tensor([ 0.])

1.0381180047988892
0.7384867072105408
FALSE POSITIVE

tensor([ 1.])

1.565530776977539
0.827145516872406
TRUE POSITIVE

tensor([ 1.])

1.6089540719985962
0.8332660794258118
TRUE POSITIVE

tensor([ 0.])

-4.069083213806152
0.01680579036474228
TRUE NEGATIVE

tensor([ 0.])

-5.200493812561035
0.005483605433255434
TRUE NEGATIVE

tensor([ 0.])

-10.796891212463379
2.0462599422899075e-05
TRUE NEGATIVE

tensor([ 0.])

3.570046901702881
0.9726164937019348
FALSE POSITIVE

tensor([ 0.])

0.9478980302810669
0.7206922769546509
FALSE POSITIVE

tensor([ 1.])

4.112088680267334
0.9838902354240417
TRUE POSITIVE

tensor([ 1.])

4.228627681732178
0.9856369495391846
TRUE POSITIVE

tensor([ 0.])

-1.9449385404586792
0.1251063048839569
TRUE NEGATIVE

tensor([ 0.])

-12.222395896911621
4.919022103422321e-06
TRUE NEGATIVE

tensor([ 1.])

5.081783771514893
0.9938295483589172
TRUE POSITIVE

tensor([ 1.])

2.072937488555908
0.8882449269294739
T

tensor([ 1.])

0.04865184426307678
0.5121605396270752
TRUE POSITIVE

tensor([ 1.])

2.1873528957366943
0.8991080522537231
TRUE POSITIVE

tensor([ 1.])

0.8902353048324585
0.708938717842102
TRUE POSITIVE

tensor([ 0.])

-8.107893943786621
0.00030106178019195795
TRUE NEGATIVE

tensor([ 1.])

0.04865184426307678
0.5121605396270752
TRUE POSITIVE

tensor([ 0.])

4.777464866638184
0.9916529059410095
FALSE POSITIVE

tensor([ 1.])

0.17495280504226685
0.5436269640922546
TRUE POSITIVE

tensor([ 1.])

0.04865184426307678
0.5121605396270752
TRUE POSITIVE

tensor([ 1.])

1.1492559909820557
0.7593749761581421
TRUE POSITIVE

tensor([ 1.])

1.2602473497390747
0.7790687084197998
TRUE POSITIVE

tensor([ 1.])

0.9709259271621704
0.7253040075302124
TRUE POSITIVE

tensor([ 0.])

-1.1038726568222046
0.2490149885416031
TRUE NEGATIVE

tensor([ 0.])

-6.154248237609863
0.0021199339535087347
TRUE NEGATIVE

tensor([ 0.])

-1.7633129358291626
0.14637590944766998
TRUE NEGATIVE

tensor([ 1.])

0.9808995723724365
0

In [7]:
correct = tp+tn
print(100*correct/total)

print((tp/total)*100, (fp/total)*100)
print((tn/total)*100, (fn/total)*100)


74.77876106194691
50.442477876106196 20.79646017699115
24.336283185840706 4.424778761061947
